# Big Data, Predictive Analytics and Deep Learning Workshop
## Notebook #1

Just to keep things real, let's try a simple python command:

In [ ]:
print("Hello, big data!")

In [ ]:
df_from_other_list = spark.createDataFrame(
    [('Chris',67),('Frank',70)], [‘name','score'])
df_from_other_list.show()

In [ ]:
from pyspark.sql.types import FloatType
df_from_list = spark.createDataFrame(
    [1.0,2.0,3.0,4.0,5.0], FloatType())
df_from_list.show()

In [ ]:
df = spark.createDataFrame([('Chris',[67,42]),('Frank',[70,72])],['name','scores'])

In [ ]:
df.show()

In [ ]:
from pyspark.sql.functions import explode


In [ ]:
df = df.withColumn('score',explode('scores'))

In [ ]:
df.show()

In [ ]:
import pyspark.sql.functions as F

In [ ]:
df.withColumn('good', F.when(df['score'] > 50,1).otherwise(0)).show()

Next, we're going to "mount" an Amazon Web Services (AWS) S3 bucket.  Think of this as a virtual USB key drive:  
whatever is in that bucket will be available to your Databricks notebook via the path "/mnt/umsi-data-science".   The following cell
mounts the S3 bucket that I've set up for this workshop.

**NOTE: You will need to change the value of SECRET_KEY to the value that we sent you via email.**

In [ ]:
# Change the following line
SECRET_KEY = ""

# DO NOT change the following lines
ACCESS_KEY = ""

ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "umsi-data-science-west"
MOUNT_NAME = "umsi-data-science"
try:
  dbutils.fs.unmount("/mnt/%s/" % MOUNT_NAME)
except:
  print("Could not unmount %s, but that's ok." % MOUNT_NAME)
dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
print("%s has been mounted." % MOUNT_NAME)

In [ ]:
display(dbutils.fs.ls("/mnt/umsi-data-science/data"))

In [ ]:
df = spark.read.json("/mnt/umsi-data-science/data/yelp/business.json")

In [ ]:
df.printSchema()

In [ ]:
df.createOrReplaceTempView("businesses")

sqlDF = spark.sql("SELECT * FROM businesses")
sqlDF.show()


In [ ]:
sqlDF = spark.sql("SELECT state,count(*) as c FROM businesses where attributes.ambience.hipster group by state order by c desc" )
sqlDF.show()